In [1]:
from opensea import assets,bundles,common,events
import json

# JSON keys in API CALL

In [10]:
asset = assets.get_assets(collection="cryptokitties",limit=1,verified_only=False)
print("First index asset keys")
asset[0].get_json().keys()

First index asset keys


dict_keys(['id', 'token_id', 'num_sales', 'background_color', 'image_url', 'image_preview_url', 'image_thumbnail_url', 'image_original_url', 'animation_url', 'animation_original_url', 'name', 'description', 'external_link', 'asset_contract', 'permalink', 'collection', 'decimals', 'token_metadata', 'owner', 'sell_orders', 'creator', 'traits', 'last_sale', 'top_bid', 'listing_date', 'is_presale', 'transfer_fee_payment_token', 'transfer_fee', 'related_assets', 'orders', 'auctions', 'supports_wyvern', 'top_ownerships', 'ownership', 'highest_buyer_commitment'])

In [ ]:
# In depth look at json, keys such as collection has more subkeys
json_asset = json.dumps(asset[0].get_json(), indent = 4, sort_keys=True) 
print(json_asset)

We can copy this and paste into http://jsonviewer.stack.hu/ to interactively see our variables we can use

If you take a look at opensea > models > asset.py 

``` Python
class Asset:
    def __init__(self, json_data):
        
        """
        Put useful information from json_data into their own variables under Asset class.

        :param json_data: json object returned by the opensea-api
        :type json_data: dict
        """
        # ASSET DETAILS
        self.name = json_data["name"]
        self.description = json_data["description"]
        self.token_id = json_data["token_id"]
        self.asset_url = json_data["permalink"]
        self.image_url = json_data["image_url"]
```

If we want to add say the created_date of the crypto we just need to add 
``` Python
self.created_date = json_data["created_date"]
```
and we will be able to call
``` Python
print(asset.created_date)
```

# A look into the Images from OpenSea

In [5]:
# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)
from PIL import Image
# load the image
import requests
from io import BytesIO

asset_list = assets.get_assets(collection="cryptokitties",limit=3,verified_only=True)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    url = asset.image_url
    print(url)
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    # summarize some details about the image
    print(img.format)
    print(img.mode)
    print(img.size)
    # show the image
    img.show()

TypeError: object of type 'Response' has no len()

Note that we have images with different resolutions, this is something we will need to deal with, prehaps a simple rescale (svd?)

In [ ]:
# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)

asset_list = assets.get_assets(limit=5,verified_only=True)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list

    print(asset.owner.address)
    print(asset.owner.profile_img_url)
    print(asset.owner.username)
    print(asset.owner.config)

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    print(asset.image_url)
    print(asset.get_floor_price()) # Floor price of the collection

# A look into querying 1 NFT collection

In [ ]:

# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)

# Order by sale_price to get order by last total_price

asset_list = assets.get_assets(collection="cryptokitties",limit=1, verified_only=True, token_ids=896775)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    print("i:", i)
    # print(asset.owner.address)    

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    # print(asset.image_url)
    print("price floor", asset.get_floor_price()) # Floor price of the collection

    


In [ ]:
# In depth look at json, keys such as collection has more subkeys
json_asset = json.dumps(asset_list[0].get_json(), indent = 4, sort_keys=True) 
print(json_asset)

# Looking at NFT EVENT transactions

In [ ]:
asset_list = events.get_events(event_type='successful',asset_contract_address="0x06012c8cf97BEaD5deAe237070F9587f8E7A266d", token_id=896775)
asset_list


In [ ]:
asset_list["asset_events"][0]

# Running through to see a how many kitties are currently on sale, or have a last_price

In [19]:
counter = 0 
import time
for i in range(0, 400):
    try:
        print(list(range((i*30)+1, (i*30)+31)))
        asset_list = assets.get_assets(collection="cryptokitties",limit=30,verified_only=True,token_ids=list(range((i*30)+1, (i*30)+31)))
        print("i:", i)
        print(asset_list)
        for j in range(len(asset_list)):
            asset = asset_list[j] # Get the first asset obejct from the list
            counter += 1
            print(str(asset.name), str(int(asset.last_sale)/1000000000000000000),asset.token_id, counter)
    except Exception as inst:
        print(inst)
    time.sleep(3)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
i: 0
[<opensea.models.asset.Asset object at 0x0000022B751A5D00>, <opensea.models.asset.Asset object at 0x0000022B77477250>, <opensea.models.asset.Asset object at 0x0000022B774775E0>, <opensea.models.asset.Asset object at 0x0000022B77477DC0>, <opensea.models.asset.Asset object at 0x0000022B77477F10>, <opensea.models.asset.Asset object at 0x0000022B77477E50>, <opensea.models.asset.Asset object at 0x0000022B77477D30>, <opensea.models.asset.Asset object at 0x0000022B774779A0>, <opensea.models.asset.Asset object at 0x0000022B77477550>, <opensea.models.asset.Asset object at 0x0000022B77477F70>, <opensea.models.asset.Asset object at 0x0000022B77477760>, <opensea.models.asset.Asset object at 0x0000022B774774C0>, <opensea.models.asset.Asset object at 0x0000022B751DD640>, <opensea.models.asset.Asset object at 0x0000022B751DD550>, <opensea.models.asset.Asset object at 0x0000022B751DD2B0

KeyboardInterrupt: 

In [ ]:
asset_list = assets.get_assets(collection="cryptokitties",limit=30, verified_only=True, order_by="sale_price")
for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    print("i:", i)
    # print(asset.owner.address)    

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    # print(asset.image_url)
    print(asset.token_id)
    print("price floor", asset.get_floor_price()) # Floor price of the collection

b'\n<!doctype html>\n<html lang="en">\n<head>\n  <title>Server Error (500)</title>\n</head>\n<body>\n  <h1>Server Error (500)</h1><p></p>'
b'\n<script defer src="https://static.cloudflareinsights.com/beacon.min.js" data-cf-beacon=\'{"rayId":"6a3e3cb72a364619","token":"96'
b'047490de144c8b91be74ba7605ab69","version":"2021.10.0","si":100}\'></script>\n</body>\n</html>\n'


TypeError: object of type 'Response' has no len()